<a href="https://colab.research.google.com/github/qwehoi/music_theory_code/blob/main/%E4%B8%80%E9%8D%B5%E8%BC%B8%E5%87%BA%E5%92%8C%E5%BC%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def generate_chord_v2(root, chord_base, extensions=None, bass=None):
    """
    生成和弦信息，包括根音、和弦類型、延伸音和低音。

    :param root: 根音 (e.g., 'C', 'Db', 'F#')
    :param chord_base: 和弦基本類型 (e.g., 'maj', 'min', 'dim')
    :param extensions: 延伸音 (e.g., ['7', 'b9'])
    :param bass: 和弦低音 (e.g., 'E', 'Ab')
    :return: 和弦信息字典
    """
    # 定義音階與半音距離，包括升降音
    sharp_scale = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    flat_scale = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
    enharmonics = {**dict(zip(flat_scale, sharp_scale)), **dict(zip(sharp_scale, flat_scale))}

    def get_note_idx(note, prefer_flat=False):
        scale = flat_scale if prefer_flat else sharp_scale
        if note not in scale:
            raise ValueError(f"Unsupported note: {note}")
        return scale.index(note)

    def get_note_by_steps(start_idx, steps, prefer_flat=False):
        scale = flat_scale if prefer_flat else sharp_scale
        return scale[(start_idx + steps) % 12]

    # 和弦基本類型與延伸音的定義
    chord_definitions = {
        "maj": {"intervals": [0, 4, 7], "a.k.a.": ["major", "Maj", "M"]},
        "min": {"intervals": [0, 3, 7], "a.k.a.": ["minor", "min", "-"]},
        "dim": {"intervals": [0, 3, 6], "a.k.a.": ["diminished", "dim"]},
        "aug": {"intervals": [0, 4, 8], "a.k.a.": ["augmented", "aug"]},
        "sus2": {"intervals": [0, 2, 7], "a.k.a.": ["suspended second", "sus2"]},
        "sus4": {"intervals": [0, 5, 7], "a.k.a.": ["suspended fourth", "sus4"]},
        "7": {"intervals": [0, 4, 7, 10], "a.k.a.": ["dominant seventh", "dom"]},
    }
    extension_intervals = {
        "maj7": 11,
        "b9": 13,
        "9": 14,
        "#9": 15,
        "11": 17,
        "#11": 18,
        "13": 21,
        "b13": 20,
        "#13": 22,
    }

    # 確認和弦類型
    if chord_base not in chord_definitions:
        raise ValueError(f"Unsupported chord type: {chord_base}")

    base_info = chord_definitions[chord_base]
    intervals = base_info["intervals"]
    a_k_a = base_info["a.k.a."]

    # 判斷是否優先使用降號
    prefer_flat = 'b' in root or any('b' in ext for ext in (extensions or []))

    # 根音計算
    root_idx = get_note_idx(root, prefer_flat)

    # 添加延伸音
    if extensions:
        for ext in extensions:
            if ext in extension_intervals:
                intervals.append(extension_intervals[ext])
            else:
                raise ValueError(f"Unsupported extension: {ext}")

    # 計算和弦音符
    notes = [get_note_by_steps(root_idx, step, prefer_flat) for step in intervals]
    if bass:
        bass_idx = get_note_idx(bass, prefer_flat)
        bass_note = get_note_by_steps(bass_idx, 0, prefer_flat)
        notes = [bass_note] + [note for note in notes if note != bass_note]

    # 計算間隔和半音
    half_steps = [intervals[i] - intervals[i - 1] if i > 0 else intervals[i] for i in range(len(intervals))]

    return {
        "root": root,
        "chord_type": chord_base,
        "a.k.a.": a_k_a,
        "extensions": extensions or [],
        "bass": bass,
        "notes": notes,
        "intervals": [1] + [step for step in intervals[1:]],
        "half_steps": half_steps
    }


# 測試生成和弦
examples = [
    ("F#", "7", None, None),
    ("A", "min", ["11"], None),
    ("Db", "maj", None, "Ab"),
]

for root, chord_base, extensions, bass in examples:
    try:
        chord = generate_chord_v2(root, chord_base, extensions, bass)
        print(f"{root} {chord_base}{' ' + ' '.join(chord['extensions']) if chord['extensions'] else ''}")
        print(f"a.k.a.:     {', '.join(chord['a.k.a.'])}")
        print(f"intervals:  {', '.join(map(str, chord['intervals']))}")
        print(f"half-steps: {', '.join(map(str, chord['half_steps']))}")
        print(f"notes:      {', '.join(chord['notes'])}\n")
    except ValueError as e:
        print(e)


F# 7
a.k.a.:     dominant seventh, dom
intervals:  1, 4, 7, 10
half-steps: 0, 4, 3, 3
notes:      F#, A#, C#, E

A min 11
a.k.a.:     minor, min, -
intervals:  1, 3, 7, 17
half-steps: 0, 3, 4, 10
notes:      A, C, E, D

Db maj
a.k.a.:     major, Maj, M
intervals:  1, 4, 7
half-steps: 0, 4, 3
notes:      Ab, Db, F

